# Classification of healthy and Non healthy individual on the basis of their lung sounds

`By VISHAL KUMAR`

In [37]:
#importing the necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import os

from imblearn.over_sampling import SMOTE

from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import IsolationForest
from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score,accuracy_score,roc_auc_score


In [2]:
#read test and train datasets formed after preprocessing
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [4]:
train_data

,Unnamed: 0,filename,split,Age,Gender,BMI,is_healthy,disease
0,3,103_2b2_Ar_mc_LittC2SE,train,70.0,0,33.000000,0,Asthma
1,10,105_1b1_Tc_sc_Meditron,train,7.0,0,17.558299,0,URTI
2,11,106_2b1_Pl_mc_LittC2SE,train,73.0,0,21.000000,0,COPD
3,12,106_2b1_Pr_mc_LittC2SE,train,73.0,0,21.000000,0,COPD
4,13,107_2b3_Al_mc_AKGC417L,train,75.0,0,33.700000,0,COPD
...,...,...,...,...,...,...,...,...
534,906,222_1b1_Lr_sc_Meditron,train,60.0,1,22.000000,0,COPD
535,907,222_1b1_Pr_sc_Meditron,train,60.0,1,22.000000,0,COPD
536,917,226_1b1_Al_sc_Meditron,train,4.0,1,15.741352,0,Pneumoni
537,918,226_1b1_Ll_sc_Meditron,train,4.0,1,15.741352,0,Pneumoni


In [5]:
train_data['is_healthy'].value_counts()

0    521
1     18
Name: is_healthy, dtype: int64

In [6]:
train_data['disease'].value_counts()

COPD              444
Pneumonia          34
Healthy            18
URTI               16
Bronchiectasis     14
Bronchiolitis       7
Pneumoni            3
LRTI                2
Asthma              1
Name: disease, dtype: int64

This shows the imbalance of data

### First we will load data 
We use Librosa to extract MFCC audio features to be used for training.


In [7]:
dir = os.getcwd()

X_train=[]
y_train=[]
for i in range(len(train_data)):
    # if train_data["is_healthy"][i]!=1:
    file = dir+"/"+train_data["filename"][i]+".wav"
    data_x, sampling_rate = librosa.load(file)
    mfccs = np.mean(librosa.feature.mfcc(y=data_x, sr=sampling_rate, n_mfcc=40).T,axis=0) 
    X_train.append(mfccs)
    y_train.append(train_data["is_healthy"][i])

X_train=np.array(X_train)
y_train=np.array(y_train)

In [8]:
X_test=[]
y_test=[]
for i in range(len(test_data)):
    file = dir+"/"+test_data["filename"][i]+".wav"
    data_x, sampling_rate = librosa.load(file)
    mfccs = np.mean(librosa.feature.mfcc(y=data_x, sr=sampling_rate, n_mfcc=40).T,axis=0) 
    X_test.append(mfccs)
    y_test.append(train_data["is_healthy"][i])

X_test=np.array(X_test)
y_test=np.array(y_test)

Now we will fir some classifiers


In [ ]:
#First is an ensemble technique Random Forest Classifier

In [9]:
#shuffle the data before training
X_shuffled,y_shuffled = shuffle(X_train,y_train)

In [39]:
clf = RandomForestClassifier(n_estimators=15,max_depth=2, random_state=0, bootstrap=True, oob_score=True,class_weight="balanced")
#using balanced class weights since it is an imbalanced dataset
clf.fit(X_shuffled,y_shuffled)

print("For Training Data")
print("Accuracy: ",accuracy_score(y_shuffled,clf.predict(X_shuffled)))
print("Precision: ",precision_score(y_shuffled,clf.predict(X_shuffled)))
print("Recall: ",recall_score(y_shuffled,clf.predict(X_shuffled)))
print("F1 score: ",f1_score(y_shuffled,clf.predict(X_shuffled)))
print("AUC-ROC score: ",roc_auc_score(y_shuffled,clf.predict(X_shuffled)))

print("For Test data")
print("Accuracy: ",accuracy_score(y_test,clf.predict(X_test)))
print("Precision: ",precision_score(y_test,clf.predict(X_test)))
print("Recall: ",recall_score(y_test,clf.predict(X_test)))
print("F1 score: ",f1_score(y_test,clf.predict(X_test)))
print("AUC-ROC score: ",roc_auc_score(y_test,clf.predict(X_test)))



For Training Data
Accuracy:  0.9424860853432282
Precision:  0.3673469387755102
Recall:  1.0
F1 score:  0.5373134328358209
AUC-ROC score:  0.9702495201535509
For Test data
Accuracy:  0.905511811023622
Precision:  0.0
Recall:  0.0
F1 score:  0.0
AUC-ROC score:  0.4713114754098361


We are getting poor performance on test data 
Let's try some data augmentation

In [23]:
#functions to augment data
def add_noise(data,x):
    noise = np.random.randn(len(data))
    data_noise = data + x * noise
    return data_noise

def shift(data,x):
    return np.roll(data, x)

def stretch(data, rate):
    data = librosa.effects.time_stretch(data, rate)
    return data

In [24]:
X_train=[]
y_train=[]
for i in range(len(train_data)):
    if train_data["is_healthy"][i]!=1:
        file = dir+"/"+train_data["filename"][i]+".wav"
        data_x, sampling_rate = librosa.load(file)
        mfccs = np.mean(librosa.feature.mfcc(y=data_x, sr=sampling_rate, n_mfcc=40).T,axis=0) 
        X_train.append(mfccs)
        y_train.append(train_data["is_healthy"][i])
        
    else:
        file = dir+"/"+train_data["filename"][i]+".wav"
        data_x, sampling_rate = librosa.load(file)
        mfccs = np.mean(librosa.feature.mfcc(y=data_x, sr=sampling_rate, n_mfcc=40).T,axis=0) 

        X_train.append(mfccs)
        y_train.append(train_data["is_healthy"][i])


        data_noise = add_noise(data_x,0.01)
        mfccs_noise = np.mean(librosa.feature.mfcc(y=data_noise, sr=sampling_rate, n_mfcc=40).T,axis=0) 
        X_train.append(mfccs_noise)
        y_train.append(train_data["is_healthy"][i])


        data_shift = shift(data_x,3200)
        mfccs_shift = np.mean(librosa.feature.mfcc(y=data_shift, sr=sampling_rate, n_mfcc=40).T,axis=0) 
        X_train.append(mfccs_shift)
        y_train.append(train_data["is_healthy"][i])

        data_stretch = stretch(data_x,1.2)
        mfccs_stretch = np.mean(librosa.feature.mfcc(y=data_stretch, sr=sampling_rate, n_mfcc=40).T,axis=0) 
        X_train.append(mfccs_stretch)
        y_train.append(train_data["is_healthy"][i])

        data_stretch_2 = stretch(data_x,0.8)
        mfccs_stretch_2 = np.mean(librosa.feature.mfcc(y=data_stretch_2, sr=sampling_rate, n_mfcc=40).T,axis=0) 
        X_train.append(mfccs_stretch_2)
        y_train.append(train_data["is_healthy"][i])

        
        

X_train=np.array(X_train)
y_train=np.array(y_train)

In [44]:
X_shuffled_1,y_shuffled_1 = shuffle(X_train,y_train)

clf = RandomForestClassifier(n_estimators=20,max_depth=2, random_state=0, bootstrap=True, oob_score=True,class_weight="balanced")
clf.fit(X_shuffled_1,y_shuffled_1)

print("For Training Data")
print("Accuracy: ",accuracy_score(y_shuffled_1,clf.predict(X_shuffled_1)))
print("Precision: ",precision_score(y_shuffled_1,clf.predict(X_shuffled_1)))
print("Recall: ",recall_score(y_shuffled_1,clf.predict(X_shuffled_1)))
print("F1 score: ",f1_score(y_shuffled_1,clf.predict(X_shuffled_1)))
print("AUC-ROC score: ",roc_auc_score(y_shuffled_1,clf.predict(X_shuffled_1)))

print("For Test data")
print("Accuracy: ",accuracy_score(y_test,clf.predict(X_test)))
print("Precision: ",precision_score(y_test,clf.predict(X_test)))
print("Recall: ",recall_score(y_test,clf.predict(X_test)))
print("F1 score: ",f1_score(y_test,clf.predict(X_test)))
print("AUC-ROC score: ",roc_auc_score(y_test,clf.predict(X_test)))

For Training Data
Accuracy:  0.911620294599018
Precision:  0.6285714285714286
Recall:  0.9777777777777777
F1 score:  0.7652173913043477
AUC-ROC score:  0.9389848581787161
For Test data
Accuracy:  0.8713910761154856
Precision:  0.0
Recall:  0.0
F1 score:  0.0
AUC-ROC score:  0.453551912568306


Lets try oversampling with this data

In [26]:
oversample = SMOTE(random_state=0)
X_train_over,y_train_over = oversample.fit_resample(X_train,y_train)

Z=pd.DataFrame(y_train_over,columns=["target"])
Z["target"].value_counts()

0    521
1    521
Name: target, dtype: int64

In [45]:
X_shuffled_over,y_shuffled_over= shuffle(X_train_over,y_train_over)

clf = RandomForestClassifier(n_estimators=20,max_depth=2, random_state=0, bootstrap=True, oob_score=True)
clf.fit(X_shuffled_over,y_shuffled_over)

print("For Training Data")
print("Accuracy: ",accuracy_score(y_shuffled_over,clf.predict(X_shuffled_over)))
print("Precision: ",precision_score(y_shuffled_over,clf.predict(X_shuffled_over)))
print("Recall: ",recall_score(y_shuffled_over,clf.predict(X_shuffled_over)))
print("F1 score: ",f1_score(y_shuffled_over,clf.predict(X_shuffled_over)))
print("AUC-ROC score: ",roc_auc_score(y_shuffled_over,clf.predict(X_shuffled_over)))

print("For Test data")
print("Accuracy: ",accuracy_score(y_test,clf.predict(X_test)))
print("Precision: ",precision_score(y_test,clf.predict(X_test)))
print("Recall: ",recall_score(y_test,clf.predict(X_test)))
print("F1 score: ",f1_score(y_test,clf.predict(X_test)))
print("AUC-ROC score: ",roc_auc_score(y_test,clf.predict(X_test)))


For Training Data
Accuracy:  0.9414587332053743
Precision:  0.9151624548736462
Recall:  0.9731285988483686
F1 score:  0.9432558139534883
AUC-ROC score:  0.9414587332053743
For Test data
Accuracy:  0.847769028871391
Precision:  0.022222222222222223
Recall:  0.06666666666666667
F1 score:  0.03333333333333333
AUC-ROC score:  0.473224043715847


Still the training metrics are getting better and better but the test ones aren't

Trying by adding demographic features to the inputs

In [32]:
X_train_demographic=[]
y_train_demographic=[]
for i in range(len(train_data)):
    if train_data["is_healthy"][i]!=1:
        file = dir+"/"+train_data["filename"][i]+".wav"
        data_x, sampling_rate = librosa.load(file)
        mfccs = np.mean(librosa.feature.mfcc(y=data_x, sr=sampling_rate, n_mfcc=40).T,axis=0) 
        mfccs = np.append(mfccs,np.array([train_data["Age"][i],train_data["Gender"][i],train_data["BMI"][i]]))
        X_train_demographic.append(mfccs)
        y_train_demographic.append(train_data["is_healthy"][i])
        
    else:
        file = dir+"/"+train_data["filename"][i]+".wav"
        data_x, sampling_rate = librosa.load(file)
        mfccs = np.mean(librosa.feature.mfcc(y=data_x, sr=sampling_rate, n_mfcc=40).T,axis=0) 
        mfccs = np.append(mfccs,np.array([train_data["Age"][i],train_data["Gender"][i],train_data["BMI"][i]]))

        X_train_demographic.append(mfccs)
        y_train_demographic.append(train_data["is_healthy"][i])


        data_noise = add_noise(data_x,0.01)
        mfccs_noise = np.mean(librosa.feature.mfcc(y=data_noise, sr=sampling_rate, n_mfcc=40).T,axis=0) 
        mfccs_noise = np.append(mfccs_noise,np.array([train_data["Age"][i],train_data["Gender"][i],train_data["BMI"][i]]))
        X_train_demographic.append(mfccs_noise)
        y_train_demographic.append(train_data["is_healthy"][i])


        data_shift = shift(data_x,3200)
        mfccs_shift = np.mean(librosa.feature.mfcc(y=data_shift, sr=sampling_rate, n_mfcc=40).T,axis=0)
        mfccs_shift = np.append(mfccs_shift,np.array([train_data["Age"][i],train_data["Gender"][i],train_data["BMI"][i]])) 
        X_train_demographic.append(mfccs_shift)
        y_train_demographic.append(train_data["is_healthy"][i])

        data_stretch = stretch(data_x,1.2)
        mfccs_stretch = np.mean(librosa.feature.mfcc(y=data_stretch, sr=sampling_rate, n_mfcc=40).T,axis=0)
        mfccs_stretch = np.append(mfccs_stretch,np.array([train_data["Age"][i],train_data["Gender"][i],train_data["BMI"][i]])) 
        X_train_demographic.append(mfccs_stretch)
        y_train_demographic.append(train_data["is_healthy"][i])

        data_stretch_2 = stretch(data_x,0.8)
        mfccs_stretch_2 = np.mean(librosa.feature.mfcc(y=data_stretch_2, sr=sampling_rate, n_mfcc=40).T,axis=0) 
        mfccs_stretch_2 = np.append(mfccs_stretch_2,np.array([train_data["Age"][i],train_data["Gender"][i],train_data["BMI"][i]]))
        X_train_demographic.append(mfccs_stretch_2)
        y_train_demographic.append(train_data["is_healthy"][i])

        
        

X_train_demographic=np.array(X_train_demographic)
y_train_demographic=np.array(y_train_demographic)

In [33]:
X_test_demographic=[]
y_test_demographic=[]
for i in range(len(test_data)):
    file = dir+"/"+test_data["filename"][i]+".wav"
    data_x, sampling_rate = librosa.load(file)
    mfccs = np.mean(librosa.feature.mfcc(y=data_x, sr=sampling_rate, n_mfcc=40).T,axis=0) 
    mfccs = np.append(mfccs,np.array([train_data["Age"][i],train_data["Gender"][i],train_data["BMI"][i]]))
    X_test_demographic.append(mfccs)
    y_test_demographic.append(train_data["is_healthy"][i])

X_test_demographic=np.array(X_test_demographic)
y_test_demographic=np.array(y_test_demographic)

In [34]:
X_train_demographic.shape

(611, 43)

In [49]:
X_shuffled_demographic,y_shuffled_demographic= shuffle(X_train_demographic,y_train_demographic)

clf = RandomForestClassifier(n_estimators=20,max_depth=2, random_state=0, bootstrap=True, oob_score=True,class_weight="balanced")
clf.fit(X_shuffled_demographic,y_shuffled_demographic)

print("For Training Data")
print("Accuracy: ",accuracy_score(y_shuffled_demographic,clf.predict(X_shuffled_demographic)))
print("Precision: ",precision_score(y_shuffled_demographic,clf.predict(X_shuffled_demographic)))
print("Recall: ",recall_score(y_shuffled_demographic,clf.predict(X_shuffled_demographic)))
print("F1 score: ",f1_score(y_shuffled_demographic,clf.predict(X_shuffled_demographic)))
print("AUC-ROC score: ",roc_auc_score(y_shuffled_demographic,clf.predict(X_shuffled_demographic)))

print("For Test data")
print("Accuracy: ",accuracy_score(y_test_demographic,clf.predict(X_test_demographic)))
print("Precision: ",precision_score(y_test_demographic,clf.predict(X_test_demographic)))
print("Recall: ",recall_score(y_test_demographic,clf.predict(X_test_demographic)))
print("F1 score: ",f1_score(y_test_demographic,clf.predict(X_test_demographic)))
print("AUC-ROC score: ",roc_auc_score(y_test_demographic,clf.predict(X_test_demographic)))


For Training Data
Accuracy:  0.9623567921440261
Precision:  0.8018018018018018
Recall:  0.9888888888888889
F1 score:  0.8855721393034827
AUC-ROC score:  0.9733312006824484
For Test data
Accuracy:  0.9291338582677166
Precision:  0.0
Recall:  0.0
F1 score:  0.0
AUC-ROC score:  0.48360655737704916


Nothing much happened again

To summurize its all happening as we have a very few examples of Healthy labels and the classifier is memorizing them from training so as we get such a hig recall.

Another way we can try is to try a clustering approach to detect outliers. We do this using IsolationForest

In [50]:
model = IsolationForest(contamination=0.04)
model.fit(X_shuffled[y_shuffled==0])

IsolationForest(contamination=0.04)

In [52]:
res = model.predict(X_test)
res[res==1]=0
res[res==-1]=1
print("Accuracy: ",accuracy_score(y_test,res))
print("Precision: ",precision_score(y_test,res))
print("Recall: ",recall_score(y_test,res))
print("F1 score: ",f1_score(y_test,res))
print("AUC-ROC score: ",roc_auc_score(y_test,res))

Accuracy:  0.931758530183727
Precision:  0.0
Recall:  0.0
F1 score:  0.0
AUC-ROC score:  0.4849726775956284


We can try neural networks approach for this problem but due to the less data available it wont be learning much anyway.
But lets try...

In [54]:
import torch
from torch import nn
from torch.utils.data import DataLoader,Dataset
from tqdm import tqdm


In [55]:
#dataset loader 
class CustomDataset(Dataset):
    def __init__(self,data,label):
        self.data = data
        self.label = label
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        data = torch.tensor(self.data[idx],dtype=torch.float32)
        label = torch.tensor(self.label[idx],dtype=torch.float32)
        return data,label

In [ ]:
GRU_train_over = np.array(X_shuffled_demographic).reshape(X_shuffled_demographic.shape[0],1,X_shuffled_demographic.shape[1])
GRU_test = np.array(X_test_demographic).reshape(X_test_demographic.shape[0],1,X_test_demographic.shape[1])
print(GRU_train_over.shape,GRU_test.shape)

In [56]:
train_set = CustomDataset(X_shuffled_demographic,y_shuffled_demographic)
test_set = CustomDataset(X_test_demographic,y_test_demographic)

In [57]:
train_loader = torch.utils.data.DataLoader(train_set,batch_size=8)
valid_loader = torch.utils.data.DataLoader(test_set,batch_size=8)

In [58]:
X_shuffled_demographic.shape

(611, 43)

In [59]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cuda device


In [63]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(43, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128,32),
            nn.ReLU(),
            nn.Linear(32,8),
            nn.ReLU(),
            nn.Linear(8,1),
            nn.Sigmoid()

        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [64]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=43, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=32, bias=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=8, bias=True)
    (7): ReLU()
    (8): Linear(in_features=8, out_features=1, bias=True)
    (9): Sigmoid()
  )
)


In [70]:
X = torch.rand(1,43, device=device)
logits = model(X)
# pred_probab = nn.Softmax(dim=1)(logits)
# y_pred = pred_probab.argmax(1)
print(f"Predicted class: {logits}")

Predicted class: tensor([[0.3943]], device='cuda:0', grad_fn=<SigmoidBackward0>)


In [73]:
n_epochs = 50
valid_max_acc = 0
train_acc = torch.zeros(n_epochs)
train_loss = torch.zeros(n_epochs)
valid_acc = torch.zeros(n_epochs)
valid_loss = torch.zeros(n_epochs)
model = NeuralNetwork()
device = ('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=0.005,momentum=0.9)
criterion = torch.nn.BCELoss()


In [82]:

for e in range(0, n_epochs):
    model.train()
    for data, labels in tqdm(train_loader):
        data, labels = data.to(device).float(), labels.to(device).float()
        
        optimizer.zero_grad()
        logits = model(data)
        labels = labels.unsqueeze(0)
        loss = criterion(logits.T, labels)
        loss.backward()
        optimizer.step()
        
        train_loss[e] += loss.item()

        logits[logits>=0.5]=1
        logits[logits<0.5]=0

        # softmax = logits.softmax(dim=1)
        # argmax = softmax.argmax(1)
        equals = (logits == labels)

        train_acc[e] += torch.mean(equals.type(torch.float)).detach().cpu()


    train_loss[e] /= len(train_loader)
    train_acc[e] /= len(train_loader)
    model.eval()
    with torch.no_grad():
        for data , labels in tqdm(valid_loader):
            data, labels = data.to(device).float(), labels.to(device).float()
            
            logits = model(data)
            labels = labels.unsqueeze(0)
            loss = criterion(logits.T,labels)
            
            valid_loss[e] += loss.item()
            logits[logits>=0.5]=1
            logits[logits<0.5]=0
            
            # softmax = logits.softmax(dim=1)
            # argmax = softmax.argmax(1)
            equals = (logits == labels)
            
            valid_acc[e] += torch.mean(equals.type(torch.float)).detach().cpu()

        valid_loss[e] /= len(valid_loader)
        valid_acc[e] /= len(valid_loader)
    print('Epoch: {} \tTraining acc: {:.6f} \tTraining Loss: {:.6f}'.format(
        e+1, train_acc[e], train_loss[e]))
    
    print('Epoch: {} \tValidation acc: {:.6f} \tValidation Loss: {:.6f}'.format(
        e+1, valid_acc[e], valid_loss[e]))
        

100%|██████████| 48/48 [00:00<00:00, 601.61it/s]


Epoch: 1 	Training acc: 0.816704 	Training Loss: 0.173545
Epoch: 1 	Validation acc: 0.923828 	Validation Loss: 0.065926


100%|██████████| 48/48 [00:00<00:00, 659.29it/s]


Epoch: 2 	Training acc: 0.733676 	Training Loss: 0.172156
Epoch: 2 	Validation acc: 0.941406 	Validation Loss: 0.139932


100%|██████████| 48/48 [00:00<00:00, 982.21it/s]


...

Epoch: 48 	Training acc: 0.851190 	Training Loss: 0.420926
Epoch: 48 	Validation acc: 0.960938 	Validation Loss: 0.228038


100%|██████████| 48/48 [00:00<00:00, 1002.65it/s]


Epoch: 49 	Training acc: 0.851190 	Training Loss: 0.420926
Epoch: 49 	Validation acc: 0.960938 	Validation Loss: 0.228038


100%|██████████| 48/48 [00:00<00:00, 982.22it/s]

Epoch: 50 	Training acc: 0.851190 	Training Loss: 0.420926
Epoch: 50 	Validation acc: 0.960938 	Validation Loss: 0.228038


Another approach one can try is to use RNN such as LSTM or GRU for training as they are quite suited for with sequence data.
I dont have enough knowledge og them and hence need some more time to delve into that.